# Lightweight Fine-Tuning Project

### TODO: In this cell, describe your choices for each of the following

**PEFT technique:**
The PEFT (Parameter-Efficient Fine-Tuning) technique used in this notebook is the LoRA (Low-Rank Adaptation) technique. LoRA reduces the number of trainable parameters by introducing trainable low-rank matrices into each layer of the Transformer model, making fine-tuning more efficient.

**Model:**
The base model used is `distilbert-base-uncased`, a smaller and faster variant of BERT. This model is pretrained on a large corpus of text and fine-tuned for sequence classification tasks, specifically binary sentiment analysis in this case (positive or negative sentiment).

**Evaluation approach:**
The evaluation approach utilizes the HuggingFace `Trainer` class, which handles the training and evaluation loops. The `compute_metrics` function calculates the accuracy of the model's predictions on the test dataset. The evaluation strategy is set to evaluate at the end of each epoch, and the best model based on evaluation metrics is saved.

**Fine-tuning dataset:**
The fine-tuning dataset is the IMDB movie reviews dataset, which consists of 500 examples each for training and testing (thinned out for faster execution in this example). The dataset is tokenized using the `distilbert-base-uncased` tokenizer, and the processed dataset is used for both training and evaluation.


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [5]:
# Import the datasets and transformers packages

from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    return tokenizer(examples['text'], truncation=True)


tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)


# Check that we tokenized the examples properly
assert tokenized_ds["train"][0]["input_ids"][:5] == [101, 2045, 2003, 2053, 7189]

# Show the first example of the tokenized training set
print(tokenized_ds["train"][0]["input_ids"])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

[101, 2045, 2003, 2053, 7189, 2012, 2035, 2090, 3481, 3771, 1998, 6337, 2099, 2021, 1996, 2755, 2008, 2119, 2024, 2610, 2186, 2055, 6355, 6997, 1012, 6337, 2099, 3504, 15594, 2100, 1010, 3481, 3771, 3504, 4438, 1012, 6337, 2099, 14811, 2024, 3243, 3722, 1012, 3481, 3771, 1005, 1055, 5436, 2024, 2521, 2062, 8552, 1012, 1012, 1012, 3481, 3771, 3504, 2062, 2066, 3539, 8343, 1010, 2065, 2057, 2031, 2000, 3962, 12319, 1012, 1012, 1012, 1996, 2364, 2839, 2003, 5410, 1998, 6881, 2080, 1010, 2021, 2031, 1000, 17936, 6767, 7054, 3401, 1000, 1012, 2111, 2066, 2000, 12826, 1010, 2000, 3648, 1010, 2000, 16157, 1012, 2129, 2055, 2074, 9107, 1029, 6057, 2518, 2205, 1010, 2111, 3015, 3481, 3771, 3504, 2137, 2021, 1010, 2006, 1996, 2060, 2192, 1010, 9177, 2027, 9544, 2137, 2186, 1006, 999, 999, 999, 1007, 1012, 2672, 2009, 1005, 1055, 1996, 2653, 1010, 2030, 1996, 4382, 1010, 2021, 1045, 2228, 2023, 2186, 2003, 2062, 2394, 2084, 2137, 1012, 2011, 1996, 2126, 1010, 1996, 5889, 2024, 2428, 2204, 1998, 6

In [7]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import numpy as np

# Load the pretrained model
pretrained_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

# Define the compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Define data collator
data_collator = DataCollatorWithPadding(tokenizer, padding=True)

# Set up the Trainer for evaluation
pretrained_trainer = Trainer(
    model=pretrained_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        per_device_eval_batch_size=2,
    ),
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Evaluate the pretrained model
pretrained_eval_results = pretrained_trainer.evaluate()
print("Pretrained model evaluation results:", pretrained_eval_results)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Pretrained model evaluation results: {'eval_loss': 0.6976129412651062, 'eval_accuracy': 0.508, 'eval_runtime': 5.9444, 'eval_samples_per_second': 84.112, 'eval_steps_per_second': 42.056}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [9]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    inference_mode=False,
    target_modules=["q_lin", "v_lin"],
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from peft import get_peft_model

lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

trainable params: 1,331,716 || all params: 67,694,596 || trainable%: 1.967241225577297


In [12]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=0.5e-3,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=4,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer, padding=True, ),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.501615,0.858000
2,0.511000,0.739129,0.844000
3,0.511000,0.857850,0.860000
4,0.166300,0.870701,0.844000


Checkpoint destination directory ./data/sentiment_analysis/checkpoint-250 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-750 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1000, training_loss=0.33863267517089846, metrics={'train_runtime': 78.9909, 'train_samples_per_second': 25.319, 'train_steps_per_second': 12.66, 'total_flos': 181874514583440.0, 'train_loss': 0.33863267517089846, 'epoch': 4.0})

In [13]:
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [14]:
from peft import PeftModelForSequenceClassification
lora_model = PeftModelForSequenceClassification.from_pretrained(model, "gpt-lora")

In [15]:
lora_trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        per_device_eval_batch_size=2,
    ),
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer, padding=True, ),
    compute_metrics=compute_metrics,
)

# Evaluate the model
eval_results = trainer.evaluate()

print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.5016147494316101, 'eval_accuracy': 0.858, 'eval_runtime': 5.9828, 'eval_samples_per_second': 83.573, 'eval_steps_per_second': 41.787, 'epoch': 4.0}


In [17]:
# Compare both models
pretrained_accuracy = pretrained_eval_results['eval_accuracy']
peft_accuracy = eval_results['eval_accuracy']

print(f"Pretrained Model Accuracy: {pretrained_accuracy}")
print(f"PEFT Model Accuracy: {peft_accuracy}")
print(f"Improvement: {peft_accuracy - pretrained_accuracy}")

Pretrained Model Accuracy: 0.508
PEFT Model Accuracy: 0.858
Improvement: 0.35


### Conclusion

The pretrained model achieved an accuracy of **0.508**.
The fine-tuned PEFT model achieved an accuracy of **0.858**, showing an improvement of **0.35**.
This demonstrates the effectiveness of the LoRA technique in enhancing model performance with reduced computational resources.
